In [1]:
from services.data_service import load_site_data, load_coef_prior_samples
from solvers.stan import _gamma_reg_data, _prior_hyperparams
import numpy as np

Importing MCMC Module: mcmc_sampling_original.py


In [2]:
# Load sites' data
(
    zbar_2017,
    gamma_vals,
    z_2017,
    forestArea_2017_ha,
    theta_vals,
    gamma_coe,
    theta_coe,
    gamma_vcov_array,
    theta_vcov_array,
    site_theta_2017_df,
    gamma_data,
) = load_site_data(
    10,
    norm_fac=1e11,
)

Data successfully loaded from '/Users/patriciohernandez/Projects/project-amazon/data/hmc/hmc_10SitesModel.csv'


In [3]:
gamma_data

,id,X1,log_historical_precip,log_historical_temp,log_lat,log_lon,log_co2e_ha_2017
0,1,1.0,1.878234,0.143973,0.538545,-2.053358,6.402935
1,5,1.0,1.878234,0.143973,0.538545,-2.053358,6.402935
2,1,1.0,2.231335,0.756389,0.737510,-1.713487,6.280606
3,2,1.0,2.231335,0.756389,0.737510,-1.713487,6.280606
4,1,1.0,2.632470,0.058021,0.899028,-2.063787,6.351144
...,...,...,...,...,...,...,...
706,10,1.0,-0.787115,-2.680370,-1.674977,0.076929,5.858993
707,10,1.0,-0.977783,0.007994,-1.341355,0.359586,5.757545
708,10,1.0,-1.313858,-1.150901,-1.596220,0.281716,5.722272
709,10,1.0,-1.323875,-0.029895,-1.492230,0.378806,5.731415


In [4]:
y_gamma, X_gamma, N_gamma, K_gamma, G_gamma = _gamma_reg_data(10, gamma_data)

In [5]:
N_gamma

689

In [6]:
I = _prior_hyperparams(y_gamma, X_gamma, "gamma")["inv_Lambda_gamma"]

In [7]:
mu = _prior_hyperparams(y_gamma, X_gamma, "gamma")["mu_gamma"]

In [8]:
mu

array([ 5.97269898,  0.06393967, -0.14306778,  0.19394488, -0.28280561])

In [ ]:
gamma_coe

In [ ]:
np.cov(beta_gamma_prior_samples).shape

In [ ]:
gamma_fitted(gamma_coe, gamma_data)

In [ ]:
gamma_fitted(mu, gamma_data)

In [ ]:
_, beta_gamma_prior_samples = load_coef_prior_samples()

In [ ]:
def gamma_fitted(gamma_coe, gamma_dataframe):
    # Copy df
    gamma_data = gamma_dataframe.copy()

    # Compute fitted values
    gamma_data["fitted_value"] = np.exp(
        (gamma_data.iloc[:, 1:6] * gamma_coe).sum(axis=1)
    )

    # Subset columns
    gamma_data = gamma_data[["id", "fitted_value"]]

    # Group by id and compute weighted mean for each group
    result = (
        gamma_data.groupby("id")["fitted_value"]
        .mean()
        .reset_index(name="gamma2017_Sites")
    )
    return result["gamma2017_Sites"].to_numpy()

In [ ]:
def new_gamma_fitted(gamma_coe, gamma_dataframe):
    # Copy df
    gamma_data = gamma_dataframe.copy()

    # Compute fitted values
    fitted = np.exp((gamma_data.iloc[:, 1:6] * gamma_coe).sum(axis=1))

    # Averaging matrix
    G = np.array([(gamma_data["id"].to_numpy() == i).astype(int) for i in range(1, 11)])
    G = G / G.sum(axis=1, keepdims=True)
    print(G)
    return G @ fitted

In [ ]:
gamma_fitted(gamma_coe, gamma_data)

In [ ]:
new_gamma_fitted(gamma_coe, gamma_data)

In [ ]:
def theta_fitted(theta_coe, theta_dataframe):
    # Copy df
    theta_data = theta_dataframe.copy()

    # Compute fitted values
    theta_data["fitted_value"] = np.exp(
        (theta_data.iloc[:, 1:9] * theta_coe).sum(axis=1)
    )

    # Subset and filter
    theta_data = theta_data[["id", "zbar_2017_muni", "fitted_value"]]
    theta_data = theta_data[theta_data["zbar_2017_muni"].notna()]

    # Weighted average function
    aux_price_2017 = 44.9736197781184

    def weighted_mean(group):
        return np.average(
            group["fitted_value"] / aux_price_2017, weights=group["zbar_2017_muni"]
        )

    # Take weighted average
    result = (
        theta_data.groupby("id")
        .apply(weighted_mean)
        .reset_index(name="theta2017_Sites")
    )

    # Return as numpy array
    return result["theta2017_Sites"].to_numpy()

In [ ]:
def new_theta_fitted(theta_coe, theta_dataframe):
    # Copy df
    theta_data = theta_dataframe.copy()

    # Filter out NA
    theta_data = theta_data[theta_data["zbar_2017_muni"].notna()]

    # Compute fitted values
    fitted = np.exp((theta_data.iloc[:, 1:9] * theta_coe).sum(axis=1))

    # Weighted average function
    aux_price_2017 = 44.9736197781184

    # Averaging matrix
    G = np.array([(theta_data["id"].to_numpy() == i).astype(int) for i in range(1, 11)])

    # Multily by weights
    G = theta_data["zbar_2017_muni"].to_numpy() * G
    G = G / G.sum(axis=1, keepdims=True)

    print(G.shape)
    return (G @ fitted) / aux_price_2017

In [ ]:
theta_fitted(theta_coe=theta_coe, theta_dataframe=site_theta_2017_df).flatten()

In [ ]:
new_theta_fitted(theta_coe=theta_coe, theta_dataframe=site_theta_2017_df).flatten()